# Extracting radiomics features

Using the pyradimocs package, extracted nodule volumes and associated segmentations we extract radiomics features

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from radiomics import featureextractor, getTestCase
import SimpleITK as sitk
import numpy as np
import glob
from pathlib import Path

In [2]:
%load_ext autoreload
%autoreload 2
from LIDC_dataset import LIDC

In [3]:
data_path = "/home/lbrocki/ConRad/dataset"
# define features to be extracted
feature_list = ["glrlm", "shape", "firstorder", "glcm", "glszm", "ngtdm", "gldm"]

In [4]:
data = LIDC(data_dir = data_path, 
            train_mode=False,
            apply_mask=False,
            return_mask=True,
            full_vol=True)

In [5]:
# Convert .pt files to .nrrd to make it readable for PyRadiomics

In [6]:
Path(f"{data_path}/crops_pyradiomics").mkdir(parents=True, exist_ok=True)
Path(f"{data_path}/masks_pyradiomics").mkdir(parents=True, exist_ok=True)
Path(f"{data_path}/radiomics_features").mkdir(parents=True, exist_ok=True)

for i in range(len(data)):
    image = np.array(data[i][0][0]*255).astype(np.uint8)
    mask = np.array(data[i][2][0]*255).astype(np.uint8)

    img = sitk.GetImageFromArray(image)
    msk = sitk.GetImageFromArray(mask)


    img_path = f"{data_path}/crops_pyradiomics/{str(i+1).zfill(4)}.nrrd"
    msk_path = f"{data_path}/masks_pyradiomics/{str(i+1).zfill(4)}.nrrd"

    sitk.WriteImage(img, img_path)
    sitk.WriteImage(msk, msk_path)

In [7]:
imgs_path = f"{data_path}/crops_pyradiomics/*"
masks_path = f"{data_path}/masks_pyradiomics/*"

imgs = glob.glob(imgs_path)
masks = glob.glob(masks_path)

imgs.sort()
masks.sort()

In [8]:
settings = {"label":255}
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
for feat in feature_list:
    print(feat)
    extractor.disableAllFeatures()
    extractor.enableFeatureClassByName(feat)
    # this feature vector is only used to extract the relevant feature names
    feature_vector = extractor.execute(imgs[0], masks[0])
    feature_names = list(feature_vector.keys())
    shape_feature_names = [x for x in feature_names if x.startswith(f"original_{feat}")]
    
    shape_features_list = []
    for i in range(len(imgs)):
        try:
            feature_vector = extractor.execute(imgs[i], masks[i])
            shape_features = [feature_vector[x] for x in shape_feature_names]
            shape_features_list.append(shape_features)
        except Exception as e:
            print(i, e)
    df = pd.DataFrame(shape_features_list, columns = shape_feature_names)
    df.to_pickle(f"{data_path}/radiomics_features/{feat}_radiomic_df.pkl")

glrlm
shape
firstorder


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


glcm


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

glszm
ngtdm
gldm
